# TP7 & 8 - Artifical Intelligence

#### Marlon KUQI & Théo MARIE

### 1 - Basic reinforcement learning example, openAI Gym environment


#### Without reinforcement learning

In [41]:
import gym
from time import sleep
from IPython.display import clear_output

# Creating thr env
env = gym.make("Taxi-v2").env

env.s = 18


# Setting the number of iterations, penalties and reward to zero,
epochs = 0
penalties, reward = 0, 0

frames = []

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1

    # Put each rendered frame into the dictionary for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
    }
    )

    epochs += 1

print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

# Printing all the possible actions, states, rewards.
def renderFrames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
renderFrames(frames)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 14
State: 410
Action: 5
Reward: 20


#### With a Deep Q-Network

In [30]:
import gym
import numpy as np
import random
from IPython.display import clear_output

# Init Taxi-V2 Env
env = gym.make("Taxi-v2").env

# Init arbitary values
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1


all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    # Init Vars
    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        if random.uniform(0, 1) < epsilon:
            # Check the action space
            action = env.action_space.sample()
        else:
            # Check the learned values
            action = np.argmax(q_table[state])

        next_state, reward, done, info = env.step(action)

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        # Update the new value
        new_value = (1 - alpha) * old_value + alpha * \
            (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print("Episode: {i}")

print("Training finished.")


Episode: {i}
Training finished.


In [31]:
# Printing q_table
q_table = pd.DataFrame(q_table)
q_table.head(329)

# north, east, south, west, pickup, drop

,0,1,2,3,4,5
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-2.273252,-2.122086,-2.273252,-2.122086,-1.870144,-11.122086
2,-1.870144,-1.450240,-1.870144,-1.450240,-0.750400,-10.450240
3,-2.363951,-2.273252,-2.363951,-2.273252,-2.122086,-11.273251
4,-2.470613,-2.472735,-2.470613,-2.473860,-9.859498,-9.270796
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,-2.489421,-2.491102,-2.489421,-2.490869,-9.420077,-8.978514
7,-2.451022,-2.460519,-2.451022,-2.462528,-8.078792,-9.017694
8,-2.273252,-2.323612,-2.340470,-2.348652,-7.936824,-6.341405
9,-2.470613,-2.476088,-2.484720,-2.478669,-10.282438,-10.310351
